In [1]:
import pandas as pd
import requests
import datetime

import env

In [7]:
search_base_url = 'https://www.googleapis.com/youtube/v3/search'
videos_base_url = 'https://www.googleapis.com/youtube/v3/videos'

search_params = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video',
  'maxResults': 20,
  'fields': 'items(id/videoId, snippet(publishedAt,title))'
}
videos_params = {
  'key': env.token,
  'part': 'statistics',
  'fields': 'items/statistics'
}

search_keywords = ['영화', '명장면', '리뷰', '해석', '의미']
columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']

tokenLimit = len(env.token)-1

In [8]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [61]:
startIndex = 65
endIndex = len(movieNames)

def get_video_list():
  columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']
  video_list = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[startIndex:endIndex]:
    print(movieName, len(video_list))
    for keyword in search_keywords:
      # parameter 설정
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      search_params['q'] = q
      search_params['publishedAfter'] = publishedAfter+'T00:00:00Z'
      search_params['publishedBefore'] = publishedBefore+'T00:00:00Z'

      res_search = requests.get(search_base_url, search_params).json()

      # 네트워크 오류 발생 시
      while 'error' in res_search.keys():
        print('[ERROR] search requests ', res_search)
        tokenNum += 1
        if tokenNum > tokenLimit: 
          print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
          return video_list

        print('[HTTPError] token 교체')
        search_params['key'] = env.token[tokenNum]
        res_search = requests.get(search_base_url, search_params).json()

      # 응답 데이터 가공
      if 'items' in res_search.keys():
        for item in res_search['items']:
          result_video_ids = list(video_list.loc[video_list['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in result_video_ids:
            videos_params['id'] = item['id']['videoId']
            res_videos = requests.get(videos_base_url, videos_params).json()

            # 네트워크 오류 발생 시
            while 'error' in res_videos.keys():
              print('[ERROR] video requests ', res_videos)
              tokenNum += 1
              if tokenNum > tokenLimit: 
                print('[Error] token 오늘 할당량 끝났다~~', movieName)
                return video_list

              print('[HTTPError] token 교체')
              videos_params['key'] = env.token[tokenNum]
              res_videos = requests.get(videos_base_url, videos_params).json()

            if 'items' in res_videos.keys():
              try:
                video_statistics = res_videos['items'][0]['statistics']
                item_data = [[
                  item['id']['videoId'], 
                  movieName, 
                  q, 
                  item['snippet']['title'], 
                  int(video_statistics['viewCount']) if 'viewCount' in video_statistics else 0,
                  int(video_statistics['likeCount']) if 'likeCount' in video_statistics else 0,
                  int(video_statistics['commentCount']) if 'commentCount' in video_statistics else 0,
                  item['snippet']['publishedAt'], 
                ]]
                video_list = video_list.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
              except Exception as e:
                print('[ERROR] ', e, movieName, res_videos)
                return video_list


  return video_list

In [62]:
video_list = get_video_list()

주먹왕 랄프 2: 인터넷 속으로 0
걸캅스 37
사자 69
아쿠아맨 105
드래곤 길들이기 3 139
나의 특별한 형제 171
어스 205
말레피센트 2 247
[ERROR] search requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
[ERROR] search requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
[ERROR] search requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you

In [65]:
lastMovieName = '미드웨이'
result = video_list.loc[video_list['영화 제목'] != lastMovieName,]
result = result.set_index('영상 id')

In [67]:
endIndex = movieNames.index(lastMovieName)

In [68]:
result.to_csv(
  ''.join(['./videoList/movieVideoList_', str(startIndex), '_', str(endIndex-1),'.csv']), 
  encoding='utf-8-sig')

In [69]:
top = lambda x: x.sort_values(by='조회수', ascending=False)[:10]
filtered_result = result.groupby('영화 제목').apply(top)
filtered_result

영화 제목           검색 키워드  \
영화 제목        영상 id                                        
#살아있다        wm06nhjoAQ8         #살아있다         #살아있다 영화   
             JbUo9nId6hA         #살아있다         #살아있다 영화   
             zBBuVe1tRwk         #살아있다         #살아있다 영화   
             LRE0y_0-b4E         #살아있다         #살아있다 영화   
             71w4EvqFydg         #살아있다         #살아있다 영화   
...                                ...              ...   
힘을 내요, 미스터 리 8pBV8LP0JP4  힘을 내요, 미스터 리  힘을 내요, 미스터 리 영화   
             -U8VhdMogPU  힘을 내요, 미스터 리  힘을 내요, 미스터 리 영화   
             I9pulIZ1-Eo  힘을 내요, 미스터 리  힘을 내요, 미스터 리 영화   
             58KliCgVxjw  힘을 내요, 미스터 리  힘을 내요, 미스터 리 영화   
             MEgNIM0Whak  힘을 내요, 미스터 리  힘을 내요, 미스터 리 영화   

                                                                      영상 제목  \
영화 제목        영상 id                                                            
#살아있다        wm06nhjoAQ8                  그냥 죽지 그랬냐, 좀비 속에서 먹방찍기... 살아있다 리뷰   
             JbUo9nId6hA     아.. 이건 아니잖아... 고민 없이 만든 한국형 좀비영화의 한계: #살아있다 리뷰   
             zBBuVe1tRwk  #음악추천 Inni - Sail / 영화 살아있다 OST (alive Soundtr...   
             LRE0y_0-b4E     《살아있다》 개봉 첫날 7000원 내고 보고 왔습니다 (개봉 첫날 가장 빠른 리뷰)   
             71w4EvqFydg                                   #살아있다 OST - Sail   
...                                                                     ...   
힘을 내요, 미스터 리 8pBV8LP0JP4  [브금브금] 2019년 8월부터 9월 최신 영화 예고편 사용음악 모음 korea m...   
             -U8VhdMogPU                  (스포포함)힘을내요 미스터리 리뷰 가벼운 추석영화가 아니네?   
             I9pulIZ1-Eo                    힘을 내요 미스터리 무대인사에서 엄채영과 박해준의 가시나   
             58KliCgVxjw  코미디와 감동의 조화! [힘을내요 미스터리]의 배우 &#39;차승원&#39;, &#...   
             MEgNIM0Whak        영화 &#39;나쁜 녀석들: 더 무비&#39; 김아중  / YTN KOREAN   

                              조회수   좋아요수  댓글 수                생성된 날짜  
영화 제목        영상 id                                                    
#살아있다        wm06nhjoAQ8  1214881  14148  3485  2020-07-05T03:15:11Z  
             JbUo9nId6hA   502061   5181  2076  2020-06-29T10:00:13Z  
             zBBuVe1tRwk   249380  10777   643  2020-06-29T14:22:40Z  
             LRE0y_0-b4E   247838   2543  1096  2020-06-24T09:06:46Z  
             71w4EvqFydg   218969   5082  1041  2020-06-29T09:40:48Z  
...                           ...    ...   ...                   ...  
힘을 내요, 미스터 리 8pBV8LP0JP4     8006     60    14  2019-09-16T05:14:00Z  
             -U8VhdMogPU     4645     21     6  2019-09-12T11:06:45Z  
             I9pulIZ1-Eo     4507     48     4  2019-09-14T13:07:49Z  
             58KliCgVxjw     2922     40     1  2019-09-24T06:00:01Z  
             MEgNIM0Whak     2905     15     0  2019-09-16T09:08:12Z  

[390 rows x 7 columns]

In [70]:
filtered_result.to_csv(
  ''.join(['./filteredVideoList/movieVideoList_',str(startIndex),'_',str(movieNames.index(lastMovieName)-1),'.csv']), 
  encoding='utf-8-sig'
)